In [25]:
import pandas as pd, numpy as np
import requests, re, locale
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

In [3]:
def date_parser(date):
    f = lambda x: re.search(r'(([0-9]|[1-3][0-9]) de [a-z]{4,10} de (19|20)\d{2})', x)
    try:
        return datetime.strptime(date, r'%d de %B de %Y')
    except ValueError:
        return date_parser(f(date).group(0))
    except:
        print('Error.')

In [15]:
def scrap(url, attrs = {'class': 'remate'}):
    """
    Devuelve un df con datos de una web
    url: str - dirección web
    
    """
    r = requests.get(url = url)
    soup = BeautifulSoup(r.text, 'lxml')
    data = soup.find_all('dl', class_ = 'remate')
    data = pd.DataFrame([(e.dt.text, e.dd.div.span.text) for e in data])
    #data[1].str.extract(pat = r'([0-9]{1,3}.[0-9])').astype(float)
    #data[['nro', 'fecha']] = data[0].str.split('-', expand = True)
    data = data.agg({1 : lambda x: x.str.extract(pat = r'([0-9]{1,4}.[0-9])').astype(float),
                     0 : lambda x: x.str.split('-', expand = True)})
    data.columns = ['precio', 'titulo', 'fecha']
    #data['fecha'] = data.agg({'fecha' : date_parser})
    return data#.set_index('fecha').sort_index()

In [16]:
# Carga de datos precio rosgan y resampleo mensual OJO
url_rosgan = r'https://www.rosgan.com.ar/precios-rosgan/'
rosgan_precio = scrap(url = url_rosgan)
#rosgan_precio = pd.read_csv('csv/rosgan.csv', parse_dates = [0], index_col = [0])
#rosgan_precio = rosgan_precio.resample('1m').last().interpolate().to_period('1m')
rosgan_precio

,precio,titulo,fecha
0,341.9,Especial Expoagro,9 de marzo de 2022
1,283.3,Rosgan Nro.154,9 al 10 de febrero de 2022
2,272.0,Rosgan Nro.153,19 de enero de 2022
3,289.8,Rosgan Nro.152,15 al 16 de diciembre de 2021
4,262.6,Rosgan Nro.151,10 de noviembre de 2021
...,...,...,...
144,6.3,Remate Nro 15,11 al 13 de febrero de 2010
145,4.5,Remate Nro 14,17 al 18 de diciembre de 2009
146,3.8,Remate Nro 13,12 al 13 de noviembre de 2009
147,3.7,Remate Nro 12,14 al 16 de octubre de 2009


In [23]:
rosgan_precio['fecha'].str.extract(r'(([0-9]|[1-3][0-9]) de ([a-z]{4,10}) de (19|20)\d{2})')

,0,1,2,3
0,9 de marzo de 2022,9,marzo,20
1,10 de febrero de 2022,10,febrero,20
2,19 de enero de 2022,19,enero,20
3,16 de diciembre de 2021,16,diciembre,20
4,10 de noviembre de 2021,10,noviembre,20
...,...,...,...,...
144,13 de febrero de 2010,13,febrero,20
145,18 de diciembre de 2009,18,diciembre,20
146,13 de noviembre de 2009,13,noviembre,20
147,16 de octubre de 2009,16,octubre,20


In [32]:
datetime.strptime(rosgan_precio['fecha'][1], r'%d de %B de %Y')

ValueError: time data ' 9 al 10 de febrero de 2022' does not match format '%d de %B de %Y'

' 9 de marzo de 2022'